In [1]:
import pandas as pd
import json
from haversine import haversine
import datetime
today_str=str(datetime.datetime.now().date())

In [2]:
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json","r"))

In [3]:
demo_df=pd.read_csv("/home/jian/Docs/Household_and_Population/2016/Demo_Dataset.csv",dtype=str)
demo_df=demo_df[['ZIP_CODE','HH15','Estimate; Female: - 25 to 29 years','Estimate; Female: - 30 to 34 years',
                 'Estimate; Female: - 35 to 39 years','Estimate; Female: - 40 to 44 years',
                 'Estimate; Female: - 45 to 49 years','Estimate; Female: - 50 to 54 years']]
for col in demo_df.columns.tolist()[1:]:
    demo_df[col]=demo_df[col].astype(float)
demo_df['Female_25_54']=demo_df[['Estimate; Female: - 25 to 29 years','Estimate; Female: - 30 to 34 years',
                 'Estimate; Female: - 35 to 39 years','Estimate; Female: - 40 to 44 years',
                 'Estimate; Female: - 45 to 49 years','Estimate; Female: - 50 to 54 years']].sum(axis=1)
demo_df=demo_df[['ZIP_CODE','HH15','Female_25_54']].rename(columns={"ZIP_CODE":"zip_cd","HH15":"Households"})
demo_df['zip_cd']=demo_df['zip_cd'].apply(lambda x:x.zfill(5))

In [4]:
demo_df['zip_cd'][2]

'01002'

In [5]:
def finding_10_miles_zips(store_center,location_id,zip_for_store):
    result=pd.DataFrame()
    i=0
    for zip_5 in zip_centers.keys():    
        dist=haversine(store_center,zip_centers[zip_5],miles=True)
        if dist<=10:        
            df=pd.DataFrame({"zip_cd":zip_5,"Center_Lat":zip_centers[zip_5][0],"Center_Long":zip_centers[zip_5][1],"Distance_Mile":dist},index=[i])
            i=i+1
            result=result.append(df)
    result['Store_id']=store_location_id
    result['Store_zip']=zip_for_store
    result=result[['Store_id','Store_zip','zip_cd','Center_Lat','Center_Long','Distance_Mile']]
    result=pd.merge(result,demo_df,on="zip_cd",how="left")
    return result

In [6]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/New_TA/Zips_within_10_miles/BL_Zips_in_10_miles_"+today_str+"_store_4676.xlsx",engine="xlsxwriter")

In [7]:
# 4676: Google Map the lat long for "1702 OCEANSIDE BLVD, Oceanside, CA 92054"
store_center=(33.192385,-117.3630678) # 4676
zip_for_store="92054"
store_location_id="4676"

df=finding_10_miles_zips(store_center,store_location_id,zip_for_store)
df.to_excel(writer,store_location_id,index=False)

In [10]:
# 4677: Google Map the lat long for "1300 DISC DRIVE Sparks, NV 89436"
store_center=(39.5776446,-119.7230628) # 4677
zip_for_store="89436"
store_location_id="4677"

df=finding_10_miles_zips(store_center,store_location_id,zip_for_store)
df.to_excel(writer,store_location_id,index=False)

In [8]:
writer.save()

In [3]:
haversine((39.5776446,-119.7230628),(39.542953,-119.73506),miles=True)

2.4806868805428506